In [ ]:
# pip install pandas openpyxl

In [3]:
import pandas as pd
import json
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import PatternFill

In [4]:
# Load the JSON data from the file
try:
    with open('data1.json', 'r') as f:
        json_data = json.load(f)
except FileNotFoundError:
    print("Error: 'data.json' not found. Make sure the file is in the same directory as your notebook.")
    exit()


In [9]:
all_rows = []

# Process each day's data
for day_data in json_data['data']:
    date = day_data['date']
    for unrealised_trade in day_data['unrealised']:
        row = {
            "Date": date,
            "Contract": "-".join(map(str, unrealised_trade['contract'])),
            "Lots": unrealised_trade['lots'],
            "Daily Action": unrealised_trade['daily_action'],
            "Entry Price": unrealised_trade['debug_info']['entry_price'],
            "Closing Price": unrealised_trade['debug_info']['closing_price'],
            "Market Lot": unrealised_trade['debug_info']['market_lot'],
            "PNL Calculation": unrealised_trade['debug_info']['pnl_calculation'],
            "PNL": unrealised_trade['pnl'],
            "Type": "Unrealised"
        }
        all_rows.append(row)
    for realised_trade in day_data['realised']:
        debug_info = realised_trade.get('debug_info', {}) # Get debug_info, default to an empty dictionary if missing
        row = {
            "Date": date,
            "Contract": "-".join(map(str, realised_trade['contract'])),
            "Lots": realised_trade['lots'],
            "Daily Action": realised_trade.get('daily_action'),
            "Entry Price": debug_info.get('entry_price'),
            "Exit Price": debug_info.get('exit_price'),
            "Market Lot": debug_info.get('market_lot'),
            "PNL Calculation": debug_info.get('pnl_calculation'),
            "PNL": realised_trade['pnl'],
            "Type": "Realised"
        }
        all_rows.append(row)

In [10]:
# Create a Pandas DataFrame
df = pd.DataFrame(all_rows)

In [11]:
# Save the DataFrame to an Excel file
excel_filename = 'trading_data1.xlsx'
df.to_excel(excel_filename, index=False)

In [12]:
# Load the workbook to add color formatting
workbook = load_workbook(excel_filename)
sheet = workbook.active

In [13]:
# Define fill colors
green_fill = PatternFill(start_color='A9F5A9', end_color='A9F5A9', fill_type='solid')
red_fill = PatternFill(start_color='F5A9A9', end_color='F5A9A9', fill_type='solid')


In [14]:
# Apply color formatting to the 'PNL' column (assuming 'PNL' is the 9th column, index 8)
for row_idx, row in enumerate(sheet.iter_rows(min_row=2, values_only=False)): # Start from the second row (skipping header)
    try:
        pnl_value = row[8].value
        if isinstance(pnl_value, (int, float)):
            if pnl_value > 0:
                row[8].fill = green_fill
            elif pnl_value < 0:
                row[8].fill = red_fill
    except IndexError:
        print(f"Warning: Row {row_idx + 2} might not have a 'PNL' column.")
    except TypeError:
        print(f"Warning: Non-numeric value found in PNL column at row {row_idx + 2}.")


In [ ]:
# Save the modified Excel file
workbook.save('trading_data1_colored.xlsx')


Data successfully exported to 'trading_data.xlsx' and 'trading_data_colored.xlsx' with P&L color-coding.
